<a href="https://colab.research.google.com/github/osaeed-ds/vector-hello/blob/main/Osaeed_pgVector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **pgVector as a Vector Database**
This is a hello world exercise based on the Vector Search quickstart on the LangChain website.
https://python.langchain.com/docs/integrations/vectorstores/pgvector

The dataset did not work in the example (did not specify where to get the file) so I substituted my own dataset.



## **Prerequisites**

In [1]:
%pip install pgVector openai tiktoken langchain psycopg2-binary

  Obtaining dependency information for pgVector from https://files.pythonhosted.org/packages/fd/8c/4b469fdc27cf8538024d68b8aa40da7ebf86690f4d7f93eb118bdea174e7/pgvector-0.2.3-py2.py3-none-any.whl.metadata
  Obtaining dependency information for psycopg2-binary from https://files.pythonhosted.org/packages/0a/7c/6aaf8c3cb05d86d2c3f407b95bac0c71a43f2718e38c1091972aacb5e1b2/psycopg2_binary-2.9.9-cp310-cp310-macosx_10_9_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 7.6 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import PGVector
from langchain.document_loaders import TextLoader
from langchain.docstore.document import Document

## **Embedding Engine**
We will use Open AI

In [3]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

## **Dataset**
We will use the US Constitution as our dataset

In [4]:
!curl https://www.govinfo.gov/content/pkg/CDOC-110hdoc50/html/CDOC-110hdoc50.htm > constitution.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  291k    0  291k    0     0   610k      0 --:--:-- --:--:-- --:--:-- 92263--:--:-- --:--:-- --:--:--  618k


## **Generate Embeddings**
Use LangChain to chunk the dataset and use OpenAI for embeddings.

In [7]:
loader = TextLoader("constitution.txt")

documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=2000, chunk_overlap=10)
docs = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()

Created a chunk of size 4562, which is longer than the specified 2000
Created a chunk of size 21641, which is longer than the specified 2000
Created a chunk of size 6612, which is longer than the specified 2000
Created a chunk of size 2609, which is longer than the specified 2000
Created a chunk of size 2239, which is longer than the specified 2000
Created a chunk of size 2679, which is longer than the specified 2000
Created a chunk of size 2927, which is longer than the specified 2000
Created a chunk of size 2233, which is longer than the specified 2000
Created a chunk of size 2149, which is longer than the specified 2000
Created a chunk of size 2394, which is longer than the specified 2000
Created a chunk of size 2017, which is longer than the specified 2000
Created a chunk of size 2739, which is longer than the specified 2000
Created a chunk of size 12487, which is longer than the specified 2000
Created a chunk of size 3409, which is longer than the specified 2000
Created a chunk of

## **Connect to pgVector and load the embeddings**

In [8]:
os.environ["MYPASSWORD"] = getpass.getpass("pgVector Password:")

In [15]:
# PGVector needs the connection string to the database.
#CONNECTION_STRING = "postgresql+psycopg2://harrisonchase@localhost:5432/test3"
#CONNECTION_STRING = PG_VECTOR_URI


# # Alternatively, you can create it from enviornment variables.
import os

CONNECTION_STRING = PGVector.connection_string_from_db_params(
     driver="psycopg2",
     host="db-postgresql-nyc3-29616-do-user-14788251-0.b.db.ondigitalocean.com",
     port="25060",
     database="",
     user="",
     password=""
)

In [16]:
# The PGVector Module will try to create a table with the name of the collection.
# So, make sure that the collection name is unique and the user has the permission to create a table.

COLLECTION_NAME = "constitution_test"

db = PGVector.from_documents(
    embedding=embeddings,
    documents=docs,
    collection_name=COLLECTION_NAME,
    connection_string=CONNECTION_STRING,
)

## **Query the DB**

In [18]:
query = "What is the role of the Vice President?"
docs_with_score = db.similarity_search_with_score(query)

OperationalError: (psycopg2.OperationalError) SSL SYSCALL error: EOF detected

[SQL: SELECT langchain_pg_embedding.collection_id AS langchain_pg_embedding_collection_id, langchain_pg_embedding.embedding AS langchain_pg_embedding_embedding, langchain_pg_embedding.document AS langchain_pg_embedding_document, langchain_pg_embedding.cmetadata AS langchain_pg_embedding_cmetadata, langchain_pg_embedding.custom_id AS langchain_pg_embedding_custom_id, langchain_pg_embedding.uuid AS langchain_pg_embedding_uuid, langchain_pg_embedding.embedding <=> %(embedding_1)s AS distance 
FROM langchain_pg_embedding JOIN langchain_pg_collection ON langchain_pg_embedding.collection_id = langchain_pg_collection.uuid 
WHERE langchain_pg_embedding.collection_id = %(collection_id_1)s::UUID ORDER BY distance ASC 
 LIMIT %(param_1)s]
[parameters: {'embedding_1': '[-0.014787547697630717,-0.033946837026941935,0.00731476649798145,-0.004147887082933369,-0.029074714420103717,0.01660471723247188,-0.01121904754370516 ...

In [ ]:
for doc, score in docs_with_score:
    print("-" * 80)
    print("Score: ", score)
    print(doc.page_content)
    print("-" * 80)